In [137]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [138]:
URL = "https://www.worlddata.info/africa/nigeria/inflation-rates.php"
page = requests.get(URL)

In [139]:
soup = BeautifulSoup(page.content, 'lxml')

In [140]:
table = soup.find('table')

all_headers = [header.text for header in table.find_all('th')]
headers = []
for header in all_headers:
    header = header.replace('Ø ', "")
    headers.append(header)

results = [{headers[i]: cell.text for i, cell in enumerate(row.find_all('td'))}
           for row in table.find_all('tr')]
del results[0]


In [141]:
results
result = []
for item in results:
    item['Nigeria'] = item['Nigeria'].replace(" %", "")
    item['EU'] = item['EU'].replace(" %", "")
    item['USA'] = item['USA'].replace(" %", "")
    item['World'] = item['World'].replace(" %", "")
    result.append(item)

In [142]:
#reading the dataframe
df_1 = pd.DataFrame(result)
df_1

,Year,Nigeria,EU,USA,World
0,2021,16.95,2.55,4.70,3.42
1,2020,13.25,0.50,1.23,1.92
2,2019,11.40,1.63,1.81,2.19
3,2018,12.09,1.74,2.44,2.44
4,2017,16.52,1.43,2.13,2.19
...,...,...,...,...,...
57,1964,0.86,3.42,1.28,n/a
58,1963,-2.69,2.92,1.24,n/a
59,1962,5.27,3.55,1.20,n/a
60,1961,6.28,2.08,1.07,n/a


In [143]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Year     62 non-null     object
 1   Nigeria  62 non-null     object
 2   EU       62 non-null     object
 3   USA      62 non-null     object
 4   World    62 non-null     object
dtypes: object(5)
memory usage: 2.5+ KB


In [144]:
df_1.describe()

,Year,Nigeria,EU,USA,World
count,62,62,62,62,62
unique,62,62,59,60,41
top,2021,16.95,2.67,1.46,n/a
freq,1,1,2,2,20


In [145]:
#the unique values in information
df_1.nunique()

Year       62
Nigeria    62
EU         59
USA        60
World      41
dtype: int64

In [146]:
#confirming the null values in the data
df_1.isnull().sum()

Year       0
Nigeria    0
EU         0
USA        0
World      0
dtype: int64

In [148]:
#the number of rows and columns 
df_1.shape

(62, 5)

## Creating a Copy of The Dataset

In [150]:
df = df_1.copy()
df

,Year,Nigeria,EU,USA,World
0,2021,16.95,2.55,4.70,3.42
1,2020,13.25,0.50,1.23,1.92
2,2019,11.40,1.63,1.81,2.19
3,2018,12.09,1.74,2.44,2.44
4,2017,16.52,1.43,2.13,2.19
...,...,...,...,...,...
57,1964,0.86,3.42,1.28,n/a
58,1963,-2.69,2.92,1.24,n/a
59,1962,5.27,3.55,1.20,n/a
60,1961,6.28,2.08,1.07,n/a


### Question1. Which year did Nigeria experienced the highest inflation?